In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import filteranddeltaF as dFF_func
import plotdeltaF_angles as df_plot
import meanByangle as df_angle_mean

from scipy.stats import ttest_ind, ttest_rel, zscore
from roi_matching import process_matched_neurons, convert_mat_to_csv, plot_matched_neurons_subset

import alignment_code as df_align

import osi_calculation as osi
import gosiAnalysis as gosi
import correlationori as corr
import filtercriteria as filter_criteria
import plotly.graph_objects as go
import helper_functions as hf
import helper_functions
import snr_calculation as snr
import cross_correlation as cc

import warnings


In [ ]:
ml = "vgatnpy"
pre = "pre"
post= "post"

In [ ]:
#load the pre and post session data
pre_df = pd.read_csv(rf'pre\dFF_{ml}_pre.csv', delimiter=",", header=0, decimal='.',engine='python')
post_df = pd.read_csv(rf'post\dFF_{ml}_post.csv', delimiter=",", header=0, decimal='.',engine='python')

iscell_pre_path = rf'J:\My Drive\0-Main\1_STRESS\{ml}\pre\iscell.npy'
iscell_post_path = rf'J:\My Drive\0-Main\1_STRESS\{ml}\post\iscell.npy'
matched_rois_path = rf'J:\My Drive\0-Main\1_STRESS\{ml}\matched_rois.csv'


final_matches, rois_pre, rois_post = process_matched_neurons(matched_rois_path, iscell_pre_path, iscell_post_path)



In [ ]:
is_cell_pre_path = rf'J:\My Drive\0-Main\1_STRESS\{ml}\pre\iscell.npy'
is_cell_post_path = rf'J:\My Drive\0-Main\1_STRESS\{ml}\post\iscell.npy'
mat_file_path = rf'J:\My Drive\0-Main\1_STRESS\{ml}\matched_rois.csv'

df_match, list_pre, list_post = process_matched_neurons(mat_file_path, is_cell_pre_path, is_cell_post_path)


#conmvert the floats to int in the list
list_pre = [int(i) for i in list_pre]
list_post = [int(i) for i in list_post]

list_pre=helper_functions.convert_list_to_int_str(list_pre)
list_post=helper_functions.convert_list_to_int_str(list_post)

#filter pre_df and post_df based on matched neurons
essen = helper_functions.get_essential_columns_as_list(pre_df)

#combine essen list with list_pre
list_pre.extend(essen)
list_post.extend(essen)

#filter the df column based on the list_pre and list_post
pre_df = pre_df[list_pre]
post_df = post_df[list_post]

pre_df_seq_raw = hf.resequence_digit_columns(pre_df)
post_df_seq_raw = hf.resequence_digit_columns(post_df)

#drop the NaN rows in both df if either pre or post has NaN

def drop_columns_with_nan_rows(df1, df2):
    # Find columns with NaN in corresponding rows
    mask = (df1.isna() | df2.isna()).any()
    
    # Drop those columns from both DataFrames
    df1_cleaned = df1.loc[:, ~mask]
    df2_cleaned = df2.loc[:, ~mask]
    
    return df1_cleaned, df2_cleaned
# def drop_columns_with_nan_rows(df1, df2):
#     # Ensure both DataFrames have the same columns
#     common_columns = df1.columns.intersection(df2.columns)
#     df1 = df1[common_columns]
#     df2 = df2[common_columns]
    
#     # Create a mask for columns that have NaN in any row in either DataFrame
#     mask = pd.concat([df1.isna().any(), df2.isna().any()], axis=1).any(axis=1)
    
#     # Get the column names to keep
#     columns_to_keep = mask[~mask].index
    
#     # Select only the columns to keep in both DataFrames
#     df1_cleaned = df1[columns_to_keep]
#     df2_cleaned = df2[columns_to_keep]
    
#     return df1_cleaned, df2_cleaned

pre_df_seq, post_df_seq = drop_columns_with_nan_rows(pre_df_seq_raw, post_df_seq_raw)
pre_df_nonan , post_df_nonan = drop_columns_with_nan_rows(pre_df, post_df)

# #replace the NaN values with 0
# pre_df_seq.fillna(0, inplace=True)
# post_df_seq.fillna(0, inplace=True)

In [ ]:
#load the pre and post session data
pre_df = pd.read_csv(rf'pre\dFF_{ml}_pre.csv', delimiter=",", header=0, decimal='.',engine='python')
post_df = pd.read_csv(rf'post\dFF_{ml}_post.csv', delimiter=",", header=0, decimal='.',engine='python')

In [ ]:
df_align.plot_raster_plotly(pre_df_seq)


In [ ]:
df_align.plot_raster_plotly(post_df_seq)


In [ ]:
responsive_preneurons_df, plotly_fig = filter_criteria.criteria_plot_population_response(pre_df_seq, inclusion_criterion="IC5", z_threshold=1, secondsbefore=0.5, secondsafter=4, reorder_rois=False)


In [ ]:
responsive_postneurons_df, plotly_fig = filter_criteria.criteria_plot_population_response(post_df_seq[post_df_seq.columns], inclusion_criterion=None, z_threshold=1, secondsbefore=0.5, secondsafter=4, reorder_rois=False)


In [ ]:
a,b=filter_criteria.criteria_plot_population_response(pre_df, inclusion_criterion="IC3", z_threshold=1, secondsbefore=0.5, secondsafter=4, reorder_rois=False)

In [ ]:
import plotdeltaF_angles
plotdeltaF_angles.plot_deltaF_angles(responsive_preneurons_df,responsive_preneurons_df.columns[30:35], gaussian_sigma=5)
plotdeltaF_angles.plot_deltaF_angles(responsive_postneurons_df,responsive_postneurons_df.columns[30:35], gaussian_sigma=5)

In [ ]:
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
filter_criteria.filter_criteria(responsive_preneurons_df, inclusion_criterion=None, z_threshold=1, secondsbefore=1, secondsafter=6)  

In [ ]:
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
filter_criteria.filter_criteria(a, inclusion_criterion=None, z_threshold=1, secondsbefore=1, secondsafter=6)  

In [ ]:
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
filter_criteria.filter_criteria(pre_df_seq, inclusion_criterion=None, z_threshold=1, secondsbefore=1, secondsafter=6)  

In [ ]:
filter_criteria.filter_criteria(post_df_seq, inclusion_criterion=None, z_threshold=1, secondsbefore=1, secondsafter=6)  

In [ ]:
rois = hf.get_digit_columns(responsive_preneurons_df)

df_angle_mean.calculate_mean_by_angle(responsive_preneurons_df, output_path='meanAngle_pre.csv',Select_columns_toplot=rois)
pre_gosi_results = gosi.calculate_gosi("meanAngle_pre.csv", column_names=rois)


df_angle_mean.calculate_mean_by_angle(responsive_postneurons_df, output_path='meanAngle_post.csv',Select_columns_toplot=rois)
post_gosi_results = gosi.calculate_gosi("meanAngle_post.csv", column_names=rois)


In [ ]:
post_gosi_results.to_csv('post_gosi_results.csv', index=False)
pre_gosi_results.to_csv('pre_gosi_results.csv', index=False)

In [ ]:
gosi.plot_gosi('pre_gosi_results.csv', "post_gosi_results.csv")

In [ ]:
osi.calculate_and_plot(pre_gosi='pre_gosi_results.csv', mean_pre ='meanAngle_pre.csv',post_gosi ="post_gosi_results.csv", mean_post= "meanAngle_post.csv", osi_pre='osi_results_pre.csv',osi_post='osi_results_post.csv')

In [ ]:
snr.process_and_visualize_snr(pre_df_seq, post_df_seq)

In [ ]:
adjusted_df, fit_results = gosi.process_and_plot_gosi_curve('meanAngle_post.csv', 'post_gosi_results.csv')


In [ ]:
adjusted_df, fit_results = gosi.process_and_plot_gosi_curve('meanAngle_pre.csv', 'pre_gosi_results.csv')


In [ ]:
cc.analyze_correlations(pre_df, post_df, name_of_v="Pupil size")

In [ ]:
cc.analyze_correlations(pre_df, post_df, correlate_with='speed', name_of_v='Locomotion')

In [ ]:
plt.plot(pre_df["pupil_size"])


In [ ]:
plt.plot(pre_df["speed"])


In [ ]:
plt.plot(post_df["speed"])

In [ ]:
cc.plot_cross_correlations(pre_df)
cc.plot_cross_correlations(post_df)

In [ ]:
cc.plot_cross_correlations_around_stimuli(pre_df, correlate_with="pupil_size", name_of_v='Pupil Size', time_window=10)
cc.plot_cross_correlations_around_stimuli(post_df, correlate_with="pupil_size", name_of_v='Pupil Size', time_window=10)

In [ ]:
cc.plot_cross_correlations_around_stimuli(pre_df, correlate_with="speed", name_of_v='Locomotion', time_window=10)
cc.plot_cross_correlations_around_stimuli(post_df, correlate_with="speed", name_of_v='Locomotion', time_window=10)

In [ ]:
cc.plot_cross_correlations(pre_df, correlate_with="speed", name_of_v='Locomotion')
cc.plot_cross_correlations(post_df, correlate_with="speed", name_of_v='Locomotion')

In [ ]:
# import pandas as pd
# import numpy as np
# import seaborn as sns
# import matplotlib.pyplot as plt
# from scipy import stats
# import helper_functions as hf

# def plot_roi_correlation_comparison(df1, df2, alpha=0.05):
#     """
#     Plot the correlation matrices of two DataFrames side by side, print correlation statistics,
#     and create a bar plot comparing the percentage of significant correlations.
    
#     Args:
#     df1, df2 (pandas.DataFrame): DataFrames containing the ROI data.
#     alpha (float): Significance level for correlation tests.
    
#     Returns:
#     None, but displays plots and prints statistics.
#     """
    
#     def process_df(df):
#         roi_columns = hf.get_digit_columns(df)
#         roi_df = df[roi_columns].copy()
#         roi_df = roi_df.loc[:, (roi_df != 0).any(axis=0)]
#         roi_df = roi_df.dropna(axis=1)
#         return roi_df
    
#     roi_df1 = process_df(df1)
#     roi_df2 = process_df(df2)
    
#     if roi_df1.empty or roi_df2.empty:
#         print("Error: No valid ROI columns left after dropping zeros and NaNs.")
#         return
    
#     def compute_corr_stats(corr_matrix):
#         n = corr_matrix.shape[0]
#         tri_k = n * (n-1) // 2  # number of elements in upper triangle
        
#         # Compute p-values
#         p_values = np.zeros_like(corr_matrix)
#         for i in range(n):
#             for j in range(i+1, n):  # Only upper triangle
#                 r = corr_matrix.iloc[i, j]
#                 t = r * np.sqrt((n-2) / (1-r**2))
#                 p_values[i, j] = stats.t.sf(np.abs(t), n-2)*2
        
#         # Count significant correlations in upper triangle
#         sig_corrs = np.sum(p_values[np.triu_indices(n, k=1)] < alpha)
#         percent_sig = (sig_corrs / tri_k) * 100
        
#         # Calculate mean and median correlation from upper triangle
#         corr_values = corr_matrix.values[np.triu_indices(n, k=1)]
#         mean_corr = np.mean(corr_values)
#         median_corr = np.median(corr_values)
        
#         return sig_corrs, tri_k, percent_sig, mean_corr, median_corr
    
#     corr_matrix1 = roi_df1.corr()
#     corr_matrix2 = roi_df2.corr()
    
#     sig_corrs1, total_corrs1, percent_sig1, mean_corr1, median_corr1 = compute_corr_stats(corr_matrix1)
#     sig_corrs2, total_corrs2, percent_sig2, mean_corr2, median_corr2 = compute_corr_stats(corr_matrix2)
    
#     # T-test for proportion of significant correlations
#     def proportion_ttest(count1, nobs1, count2, nobs2):
#         p1 = count1 / nobs1
#         p2 = count2 / nobs2
#         se = np.sqrt(p1 * (1 - p1) / nobs1 + p2 * (1 - p2) / nobs2)
#         t = (p1 - p2) / se
#         df = nobs1 + nobs2 - 2
#         p_value = 2 * (1 - stats.t.cdf(np.abs(t), df))
#         return t, p_value

#     t_stat, p_value_ttest = proportion_ttest(sig_corrs1, total_corrs1, sig_corrs2, total_corrs2)
    
#     # Plotting correlation matrices
#     fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 7.5))
    
#     rwb_cmap = sns.diverging_palette(220, 20, as_cmap=True)
    
#     sns.heatmap(corr_matrix1, cmap=rwb_cmap, vmin=-1, vmax=1, center=0, ax=ax1)
#     ax1.set_title("Correlation Matrix Pre-stress", fontsize=16)
    
#     sns.heatmap(corr_matrix2, cmap=rwb_cmap, vmin=-1, vmax=1, center=0, ax=ax2)
#     ax2.set_title("Correlation Matrix Post-stress", fontsize=16)
    
#     plt.tight_layout()
#     plt.show()
    
#     # GraphPad Prism-like theme adjustments
#     plt.style.use('seaborn-white')  # Use a clean background
#     fig, ax = plt.subplots(figsize=(4, 7))
#     bar_width = 0.7
#     index = np.arange(2)

#     # Pastel colors
#     colors = ['#aec7e8', '#98df8a']  # Pastel blue and green

#     bars = plt.bar(index, [percent_sig1, percent_sig2], bar_width,
#                 alpha=0.9, color=colors, label=['Pre Stress', 'Post Stress'], edgecolor='black')

#     # Make the plot resemble GraphPad Prism's style
#     plt.ylabel('Percentage', fontsize=12, fontweight='bold')
#     plt.title('Ratio of ROIs with Significant Correlations', fontsize=14, fontweight='bold')
#     plt.xticks(index, ('Pre-stress', 'Post-stress'), fontsize=11)
#     plt.yticks(fontsize=11)
#     plt.legend(frameon=False, fontsize=11)

#     # Customize the axes and grid
#     ax.spines['top'].set_visible(False)
#     ax.spines['right'].set_visible(False)
#     ax.spines['left'].set_color('grey')
#     ax.spines['bottom'].set_color('grey')
#     ax.yaxis.grid(True, linestyle='--', which='major', color='lightgrey', alpha=0.7)  # Light horizontal grid lines

#     # Add significance bar if the difference is significant
#     if p_value_ttest < alpha:
#         y_max = max(percent_sig1, percent_sig2)
#         bar_height = y_max * 0.1
#         plt.plot([index[0], index[1]], [y_max + bar_height, y_max + bar_height], color='black', linewidth=1.5)
#         plt.text((index[0] + index[1]) / 2, y_max + bar_height * 1.1, f'*', 
#                  ha='center', va='bottom', fontsize=10)
#         ax.set_ylim(top=(y_max + bar_height * 2))  # Adjust y-axis limit to accommodate the bar

#     plt.tight_layout()
#     plt.show()
    
#     # Print statistics
#     print(f"Dataset 1:")
#     print(f"  Percentage of significant correlations: {percent_sig1:.2f}%")
#     print(f"  Mean correlation: {mean_corr1:.4f}")
#     print(f"  Median correlation: {median_corr1:.4f}")
#     print(f"\nDataset 2:")
#     print(f"  Percentage of significant correlations: {percent_sig2:.2f}%")
#     print(f"  Mean correlation: {mean_corr2:.4f}")
#     print(f"  Median correlation: {median_corr2:.4f}")
    
#     # Compute differences
#     diff_percent_sig = percent_sig2 - percent_sig1
#     diff_mean_corr = mean_corr2 - mean_corr1
#     diff_median_corr = median_corr2 - median_corr1
    
#     # Fisher's r-to-z transformation for testing difference between correlations
#     z1 = np.arctanh(mean_corr1)
#     z2 = np.arctanh(mean_corr2)
#     n1 = corr_matrix1.shape[0]
#     n2 = corr_matrix2.shape[0]
#     se_diff = np.sqrt(1/(n1-3) + 1/(n2-3))
#     z = (z2 - z1) / se_diff
#     p_value = 2 * (1 - stats.norm.cdf(abs(z)))
    
#     print(f"\nDifferences (Dataset 2 - Dataset 1):")
#     print(f"  Difference in percentage of significant correlations: {diff_percent_sig:.2f}%")
#     print(f"  Difference in mean correlation: {diff_mean_corr:.4f}")
#     print(f"  Difference in median correlation: {diff_median_corr:.4f}")
#     print(f"  P-value for difference in mean correlation: {p_value:.4f}")
#     print(f"  P-value for difference in percentage of significant correlations (t-test): {p_value_ttest:.10f}")

# # Usage example:
# plot_roi_correlation_comparison(pre_df, post_df)

In [ ]:
cc.plot_roi_correlation_comparison(pre_df, post_df)